In [ ]:
!pip install requests

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install pandas

In [ ]:
!pip install lxml

In [ ]:
!pip install selenium

In [ ]:
years = range(1994, 2024)

In [ ]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

In [ ]:
# Using BeautifulSoup to parse a single table

In [ ]:
# import pandas as pd
# from io import StringIO
# from bs4 import BeautifulSoup

# with open("mvp/1994.html") as f:
#     page = f.read()

# soup = BeautifulSoup(page, "html.parser")

# # Removing extra header
# soup.find('tr', class_="over_header").decompose()

# # Removing everything else from the page outside of the mvp table we want
# mvp_table = soup.find(id="mvp")

# mvp_1994 = pd.read_html(StringIO(str(mvp_table)))

# # Displays single data frame
# mvp_1994[0]

In [ ]:
# Parsing all of the tables
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup

dfs = []
for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find(id="mvp")
    mvp = pd.read_html(StringIO(str(mvp_table)))[0]

    # Adding a year column
    mvp["Year"] = year
    
    dfs.append(mvp)

In [ ]:
# Combining all of the dataframes
mvps = pd.concat(dfs)

In [ ]:
mvps.to_csv("mvps.csv")

In [ ]:
# Looking at stats for all players in the same year range

player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
# This page uses javascript to render all of the rows because of this we need to use selenium

In [ ]:
from selenium import webdriver

In [ ]:
driver = webdriver.Chrome()